## Installing fastText

The first step of this tutorial is to install and build fastText.

In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
!unzip v0.9.2.zip
!cd fastText-0.9.2 && make && pip install .

--2021-10-06 11:54:28--  https://github.com/facebookresearch/fastText/archive/v0.9.2.zip
Résolution de github.com (github.com)… 140.82.121.3
Connexion à github.com (github.com)|140.82.121.3|:443… connecté.
requête HTTP transmise, en attente de la réponse… 302 Found
Emplacement : https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2 [suivant]
--2021-10-06 11:54:29--  https://codeload.github.com/facebookresearch/fastText/zip/v0.9.2
Résolution de codeload.github.com (codeload.github.com)… 140.82.121.10
Connexion à codeload.github.com (codeload.github.com)|140.82.121.10|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : non indiqué [application/zip]
Enregistre : «v0.9.2.zip.1»

v0.9.2.zip.1            [    <=>             ]   4,17M  6,90MB/s    ds 0,6s    

2021-10-06 11:54:29 (6,90 MB/s) - «v0.9.2.zip.1» enregistré [4369852]

Archive:  v0.9.2.zip
5b5943c118b0ec5fb9cd8d20587de2b2d3966dfe
replace fastText-0.9.2/.circleci/cmake_test.sh? [y]es, [n]o, 

In [58]:
import fasttext

### Download directly with command line the english dataset

In [6]:
!cd fastText-0.9.2 && python3 download_model.py en

File exists. Use --overwrite to download anyway.


### Import HuggingFace

In [7]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


### Loading the dataset "IMDB"

In [19]:
from datasets import load_dataset

Using the split argument, we can split the imdb into two separate dataset.

In [83]:
train_dataset = load_dataset('imdb', split='train')
test_dataset = load_dataset('imdb', split='test')

Reusing dataset imdb (/home/token/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)
Reusing dataset imdb (/home/token/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a)


*train_dataset* is a class with two attributes :
- Features which contains two features : text and label (our x_train and our y_train)
- The number of rows in our dataset

In [21]:
print(train_dataset)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


If we take the first sample of our training dataset, we can see the first review and the label (positive or negative) according to that review.

In [22]:
print(train_dataset[0])

{'text': 'Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as "Teachers". My 35 years in the teaching profession lead me to believe that Bromwell High\'s satire is much closer to reality than is "Teachers". The scramble to survive financially, the insightful students who can see right through their pathetic teachers\' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I\'m here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn\'t!', 'label': 1}


### Preprocessing the dataset

With the review we previously saw, the review are definitely not preprocessed. There is still some html tags etc.. 
We definitely to work on those.

In [23]:
!pip install contractions
!pip install beautifulsoup4
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [36]:
import contractions
from bs4 import BeautifulSoup
import unicodedata
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer
from nltk.stem import PorterStemmer
nltk.download('all')


import tqdm
from tqdm import tqdm

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> l
Packages:
  [ ] abc................. Australian Broadcasting Commission 2006
  [ ] alpino.............. Alpino Dutch Treebank
  [ ] averaged_perceptron_tagger Averaged Perceptron Tagger
  [ ] averaged_perceptron_tagger_ru Averaged Perceptron Tagger (Russian)
  [ ] basque_grammars..... Grammars for Basque
  [ ] biocreative_ppi..... BioCreAtIvE (Critical Assessment of Information
                           Extraction Systems in Biology)
  [ ] bllip_wsj_no_aux.... BLLIP Parser: WSJ Model
  [ ] book_grammars....... Grammars from NLTK Book
  [ ] brown............... Brown Corpus
  [ ] brown_tei........... Brown Corpus (TEI XML Version)
  [ ] cess_cat............ CESS-CAT Treebank
  [

       | 
       | Downloading package abc to /home/token/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /home/token/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to
       |     /home/token/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /home/token/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /home/token/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /home/token/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /home/token/nltk_data...
       |   Unzipping corpora/cess_esp.zip.
       | Downloading package chat80 to /home/token/nltk_data...
       |   Unzipping corpora/chat80.zip.
       | Downloading package city_database to /home/token/nltk_data...
       |   Unzipping corpora/city_database

       | Downloading package webtext to /home/token/nltk_data...
       |   Unzipping corpora/webtext.zip.
       | Downloading package wordnet to /home/token/nltk_data...
       |   Package wordnet is already up-to-date!
       | Downloading package wordnet_ic to /home/token/nltk_data...
       |   Unzipping corpora/wordnet_ic.zip.
       | Downloading package words to /home/token/nltk_data...
       |   Unzipping corpora/words.zip.
       | Downloading package ycoe to /home/token/nltk_data...
       |   Unzipping corpora/ycoe.zip.
       | Downloading package rslp to /home/token/nltk_data...
       |   Unzipping stemmers/rslp.zip.
       | Downloading package maxent_treebank_pos_tagger to
       |     /home/token/nltk_data...
       |   Unzipping taggers/maxent_treebank_pos_tagger.zip.
       | Downloading package universal_tagset to
       |     /home/token/nltk_data...
       |   Unzipping taggers/universal_tagset.zip.
       | Downloading package maxent_ne_chunker to
       |     


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


In [ ]:
def strip_html_tags(text: str) -> str:
    '''
    Strip html tags from the input text.

            Parameters:
                    text (str): A text.

            Returns:
                    stripped_text (str): Text completely stripped without html tags.
    '''
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

In [ ]:
def remove_accented_chars(text: str) -> str:
    '''
    Remove accented characters from the text.

            Parameters:
                    text (str): A text.

            Returns:
                    text (str): Text which does not contains accented characters.
    '''
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [ ]:
def expand_contractions(text: str) -> str:
    '''
    Expand contractions like "you're" to "you are".

            Parameters:
                    text (str): A text.

            Returns:
                    text (str): Text with no contractions.
    '''
    return contractions.fix(text)

In [ ]:
def expand_contractions(text: str) -> str:
    '''
    Expand contractions like "you're" to "you are".

            Parameters:
                    text (str): A text.

            Returns:
                    text (str): Text with no contractions.
    '''
    return contractions.fix(text)

In [ ]:
def remove_special_characters(text: str , remove_digits: bool) -> str:
    '''
    Remove special characters from the text.

            Parameters:
                    text (str): A text.
                    remove_digits (bool): True if you want to remove digits, else False.

            Returns:
                    text (str): Text which does not contains special characters (digits, etc...).
    '''
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

In [ ]:
nltk.download('wordnet')

wordnet_lemmatizer = WordNetLemmatizer()

def lemming(text: str, lemming: bool) -> str:
    '''
    The words in our text input will be lemmatized to their root form. 
    Furthermore, the stop words and the words with the length less than 4 will be removed from the corpus.

            Parameters:
                    text (str): A text.
                    lemming (bool): True if you want the lemmitization, if not False.

            Returns:
                    text (str): Lemmed text.
    '''
    
    if (lemming):
        tokens = text.split()
        tokens = [wordnet_lemmatizer.lemmatize(word) for word in tokens]
        preprocessed_text = ' '.join(tokens)
        return preprocessed_text
    
    return text

In [ ]:
ps = PorterStemmer()

def stemming(text: str, stemming: bool) -> str:
    '''
    Stemming is the process of producing morphological variants of a root/base word.
    We will use this function to do the same for our document.

            Parameters:
                    text (str): A text.
                    lemming (bool): True if you want the stemmatization, if not False.

            Returns:
                    text (str): Stemmed text.
    '''
    if (stemming):
        tokens = text.split()
        tokens = [ps.stem(word) for word in tokens]
        preprocessed_text = ' '.join(tokens)
        return preprocessed_text
    
    return text

In [47]:
en_stop = set(nltk.corpus.stopwords.words('english'))

def stop_words(text: str, stop_words: bool) -> str:
    '''
    ????

            Parameters:
                    text (str): A text.
                    stop_words (bool): True if you want the delete them, if not False.

            Returns:
                    text (str): Text without stop words.
    '''
    if (stop_words):
        tokens = text.split()
        tokens = [word for word in tokens if word not in en_stop]
        tokens = [word for word in tokens if len(word) > 3]
        preprocessed_text = ' '.join(tokens)
        return preprocessed_text
    
    return text

In [118]:
def pre_process_document(document: str, lemm: bool, stemm: bool, sw: bool) -> str:
    '''
    Preprocess the document to remove all the unnecessary information.

            Parameters:
                    text (str): A text.

            Returns:
                    text (str): Text pre-processed.
    '''
    # strip HTML
    document = strip_html_tags(document)
    # lower case
    document = document.lower()
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    # remove accented characters
    document = remove_accented_chars(document)
    # expand contractions    
    document = expand_contractions(document)  
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, True)  
    # remove extra whitespace
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    document = lemming(document, lemm)
    document = stemming(document, stemm)
    
    document = stop_words(document, sw)

    return document


pre_process_corpus = np.vectorize(pre_process_document)

x_train = []
x_test = []

for i in range(train_dataset.num_rows):
    x_train.append(pre_process_document(train_dataset[i]['text'], False, False, False))
    

for i in range(test_dataset.num_rows):
    x_test.append(pre_process_document(test_dataset[i]['text'], False, False, False))

In [85]:
y_train = train_dataset['label']

In [86]:
y_test = test_dataset['label']

In [51]:
def create_imdb_train_file(train_dataset, y_train: list,
                           lemm: bool = False, stemm:bool = False, stop_words:bool = False):
    '''
    Create the imdb train file according to the fasttext documentation.
    
             Parameters:
                     train_dataset: Training dataset
                     y_train : Label for the training dataset
    '''
    
    if (lemm):
        f = open("imdb_train_lemmed.txt", "w")
    
    elif (stemm):
        f = open("imdb_train_stemmed.txt", "w")
        
    elif (stop_words):
        f = open("imdb_train_sw.txt", "w")        

    else:
        f = open("imdb_train.txt", "w") 
    
    for i in tqdm(range(train_dataset.num_rows)):
        if y_train[i] == 1:
            f.write("__label__positive" + " " + pre_process_document(train_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
        if y_train[i] == 0:
            f.write("__label__negative" + " " + pre_process_document(train_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
    
    f.close()

In [52]:
def create_imdb_test_file(test_dataset, y_test: list, lemm: bool = False, stemm: bool = False, stop_words: bool = False):
    '''
    Create the imdb test file according to the fasttext documentation.
    
             Parameters:
                     test_dataset: Training dataset
                     y_test: Label for the testing dataset
    '''
    if (lemm):
        f = open("imdb_test_lemmed.txt", "w")
    
    elif (stemm):
        f = open("imdb_test_stemmed.txt", "w")

    elif (stop_words):
        f = open("imdb_test_sw.txt", "w")        

    else:
        f = open("imdb_test.txt", "w") 
    
    for i in tqdm(range(test_dataset.num_rows)):
        if y_test[i] == 1:
            f.write("__label__positive" + " " + pre_process_document(test_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
        if y_test[i] == 0:
            f.write("__label__negative" + " " + pre_process_document(test_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
            
    f.close()

In [53]:
def create_imdb_train_and_validation_file(train_dataset, y_train: list, lemm: bool = False,
                                          stemm: bool = False, stop_words: bool = False):
    '''
    Create the imdb validation file according to the fasttext documentation.
    
             Parameters:
                     train_dataset: Training dataset
                     y_train: Label for the training dataset
    '''
    i = 0
    
    if (lemm):
        f = open("imdb_validation_lemmed.txt", "w")
    
    elif (stemm):
        f = open("imdb_validation_stemmed.txt", "w")

    elif (stop_words):
        f = open("imdb_validation_sw.txt", "w")        

    else:
        f = open("imdb_validation.txt", "w") 
    
    for i in tqdm(range(5000)):
        if y_train[i] == 1:
            f.write("__label__positive" + " " + pre_process_document(train_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
        if y_train[i] == 0:
            f.write("__label__negative" + " " + pre_process_document(train_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
    
    f.close()
    
    if (lemm):
        f = open("imdb_train_splited_with_the_validation_lemmed.txt", "w")
    
    elif (stemm):
        f = open("imdb_train_splited_with_the_validation_stemmed.txt", "w")

    elif (stop_words):
        f = open("imdb_train_splited_with_the_validation_sw.txt", "w")        

    else:
        f = open("imdb_train_splited_with_the_validation.txt", "w") 
    
    while i != train_dataset.num_rows:
        if y_train[i] == 1:
            f.write("__label__positive" + " " + pre_process_document(train_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
        if y_train[i] == 0:
            f.write("__label__negative" + " " + pre_process_document(train_dataset[i]['text'], lemm, stemm, stop_words) + '\n')
        i += 1
    
    f.close()

In [54]:
create_imdb_train_file(train_dataset, y_train)
create_imdb_train_and_validation_file(train_dataset, y_train)
create_imdb_test_file(test_dataset, y_test)

100%|██████████| 25000/25000 [00:13<00:00, 1812.98it/s]


## Let's train the model using fasttext.

##### 1) Lemming

In [87]:
create_imdb_train_file(train_dataset, y_train, lemm = True)
create_imdb_train_and_validation_file(train_dataset, y_train, lemm = True)
create_imdb_test_file(test_dataset, y_test, lemm = True)

100%|██████████| 25000/25000 [00:33<00:00, 744.22it/s]


In [88]:
%%time
model = fasttext.train_supervised(input="imdb_train_lemmed.txt")

CPU times: user 21.5 s, sys: 183 ms, total: 21.6 s
Wall time: 3.97 s


In [91]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.8726, Rappel: 0.8726


##### 2) Stemming

In [92]:
create_imdb_train_file(train_dataset, y_train, stemm = True)
create_imdb_train_and_validation_file(train_dataset, y_train, stemm = True)
create_imdb_test_file(test_dataset, y_test, stemm = True)

100%|██████████| 25000/25000 [01:36<00:00, 259.78it/s]


In [93]:
%%time
model = fasttext.train_supervised(input="imdb_train_stemmed.txt")

CPU times: user 20.2 s, sys: 167 ms, total: 20.4 s
Wall time: 3.58 s


In [94]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.8164, Rappel: 0.8164


##### 3) Stop Words

In [95]:
create_imdb_train_file(train_dataset, y_train, stop_words = True)
create_imdb_train_and_validation_file(train_dataset, y_train, stop_words = True)
create_imdb_test_file(test_dataset, y_test, stop_words = True)

100%|██████████| 25000/25000 [00:15<00:00, 1569.33it/s]


In [96]:
%%time
model = fasttext.train_supervised(input="imdb_train_sw.txt")

CPU times: user 9.91 s, sys: 183 ms, total: 10.1 s
Wall time: 2.08 s


In [97]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.87496, Rappel: 0.87496


##### 4) None

In [98]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt")

CPU times: user 18.5 s, sys: 276 ms, total: 18.8 s
Wall time: 3.52 s


In [99]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.87548, Rappel: 0.87548


### Making the model better

#### How about N-grams ?

In [100]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt", wordNgrams=2) 

CPU times: user 43.2 s, sys: 542 ms, total: 43.8 s
Wall time: 7.72 s


In [101]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.86576, Rappel: 0.86576


#### With more epochs and a smaller learning_rate ?

In [102]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt", lr=0.5, epoch=25, wordNgrams=2)

CPU times: user 3min 32s, sys: 1.61 s, total: 3min 33s
Wall time: 34.1 s


In [103]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.89536, Rappel: 0.89536


#### With more epochs and a smaller smaller learning_rate ?

In [104]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt", lr=0.25, epoch=25, wordNgrams=2)

CPU times: user 3min 33s, sys: 1.91 s, total: 3min 35s
Wall time: 33.9 s


In [105]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.89628, Rappel: 0.89628


#### With more epochs and a bigger learning_rate ?

In [106]:
%%time
model = fasttext.train_supervised(input="imdb_train.txt", lr=1.0, epoch=25, wordNgrams=2)

CPU times: user 3min 31s, sys: 1.7 s, total: 3min 33s
Wall time: 33.6 s


In [107]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.8952, Rappel: 0.8952


#### With the autotuneValidationfile (testing and performance purpose)

In [108]:
%%time
model = fasttext.train_supervised(input="imdb_train_splited_with_the_validation.txt", autotuneValidationFile='imdb_validation.txt', autotuneDuration=150)

CPU times: user 25min 48s, sys: 12.5 s, total: 26min
Wall time: 4min 17s


In [109]:
res = model.test("imdb_test.txt")
print("Nombre de sample: " + str(res[0]) + ", Taux de précision: " + str(res[1]) + ", Rappel: " + str(res[2]))

Nombre de sample: 25000, Taux de précision: 0.87896, Rappel: 0.87896


## Beating the baseline

###### Glove

In [110]:
!pip install zeugma

Defaulting to user installation because normal site-packages is not writeable


In [119]:
from zeugma.embeddings import EmbeddingTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [120]:
glove = EmbeddingTransformer('glove')
x_train = glove.transform(x_train)

In [121]:
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(x_train, y_train)

x_test = glove.transform(x_test)
y_pred = model.predict(x_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.77      0.76     12500
           1       0.76      0.73      0.75     12500

    accuracy                           0.75     25000
   macro avg       0.75      0.75      0.75     25000
weighted avg       0.75      0.75      0.75     25000

